In [2]:
## Importing libraries: PySimpleGUI is used for generating GUI for user input and PuLP is used for implementing linear programming 
import PySimpleGUI as sg
from pulp import Amply  ## AMPL from PuLP 
from pulp import *

## Create Solver function:

def Solve_ABCPath():
    
    ## GUI generating part:
    
    layout = [  [sg.Text('Parameters for ABCPath:'+'\n'+'Please follow below states instructions'+'\n'+' (1) Only enter capital letter from A to Y'+'\n'+' (2) Only one letter per box'+'\n'+' (3) Only numbers from 1-9 allowed in row number and column number')],
            [sg.Text('Enter constraint for column 1'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for column 2'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for column 3'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for column 4'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for column 5'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Row 1'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Row 2'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Row 3'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Row 4'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Row 5'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Diagonal 1'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Enter constraint for Diagonal 2'), sg.InputText('',size=(4,1)), sg.InputText('',size=(4,1))],
            [sg.Text('Row number of A:'), sg.InputText('',size=(4,1))],
            [sg.Text('Column number of A:'), sg.InputText('',size=(4,1))],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

    # Create the Window
    window = sg.Window('Parameter for ABCPath puzzle', layout)
    # Event Loop to process "events" and get the "values" of the inputs
    while True:
        event, values = window.read()
        if event in (None, 'Cancel','Ok'): 
            # if user closes window or clicks cancel
            break
 
    window.close()
    
    ## Letter mapping
    
    map={"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7,"H":8,"I":9,"J":10,"K":11,"L":12,"M":13,"N":14,"O":15,"P":16,"Q":17,"R":18,"S":19,"T":20,"U":21,"V":22,"W":23,"X":24,"Y":25}
    R={}
    Solution=[]
    k=0
    
    ## Create Parameter R(n,i,j) for LP:
     
    for j in range(1,6):
        for i in range(1,6):
            R[map[values[2*j-2]],i,j]=1
            R[map[values[2*j-1]],i,j]=1

    for i in range(1,6):
        for j in range(1,6):
            R[map[values[2*i+8]],i,j]=1
            R[map[values[2*i+9]],i,j]=1

    for i in range(1,6):
        R[map[values[20]],i,i]=1
        R[map[values[21]],i,i]=1
        R[map[values[22]],i,6-i]=1
        R[map[values[23]],i,6-i]=1
    
    KEYS=list(R.keys())
    
    ## Default value of R is 0:
    
    for n in range(1,26):
        for i in range(1,6):
            for j in range(1,6):
                if (n,i,j) not in KEYS:
                    R[(n,i,j)]=0
    
    ## Setting up index ranges:
    
    n=range(1,26)
    I=range(7)
    J=range(7)
    i=range(1,6)
    j=range(1,6)
    
    ## Parameter for initial position of letter A:
    
    ai=int(values[24])
    aj=int(values[25])
    
     ## Creating a LP problem using pulp's LpProblem:
        
    prob = LpProblem("ABCPath",LpMinimize)
    
    ## Decision Variable:
    
    x=LpVariable.dicts("X",(n,I,J),0,1,cat='Binary')
    
    ## Objective Function:
    
    prob+=0,"Arbitrary Objective Function"

    ## Constraints:
    
    ## Constraint 1:
    prob+= x[1][ai][aj]==1,""

    for num in n:
        for col in J:
            prob += x[num][0][col] == 0, ""

    ## Constraint 2:
    for num in n:
        for row in I:
            prob += x[num][row][0] == 0, ""

    ## Constraint 3:
    for num in n:
        for row in I:
            prob += x[num][row][6] == 0, ""

    ## Constraint 4:
    for num in n:
        for col in J:
            prob += x[num][6][col] == 0, ""

    ## Constraint 5:
    for num in n:
        prob += lpSum([x[num][row][col] for row in i for col in j]) == 1, ""

    ## Constraint 6:
    for row in i:
        for col in j:
            prob += lpSum([x[num][row][col] for num in n]) == 1,""

    ## Constraint 7:
    for num in range(2,25):
        for row in i:
            for col in j:
                prob+= x[num+1][row+1][col+1]+x[num+1][row+1][col]+x[num+1][row+1][col-1]+x[num+1][row][col+1]+x[num+1][row][col-1]+x[num+1][row-1][col+1]+x[num+1][row-1][col]+x[num+1][row-1][col-1] >= x[num][row][col] , ""

    ## Constraint 8:
    for num in range(2,26):
        for row in i:
            for col in j:
                prob+= x[num][row][col]<=R[(num,row,col)],""


    ## Solving generated LP:
    
    prob.solve()
    
    ## Solution display in grid formation:
    Solution=[]
    for num in range(1,26):
        for i in range(1,6):
            for j in range(1,6):
                if x[num][i][j].varValue==1:
                    Solution.append((i,j,num))
    Solution.sort()
    sol=[]
    for i in range(0,25):
        for key,value in map.items():
            if Solution[i][2]==value:
                sol.append(key)

    for i in range(0,25):
        print(sol[i],end=" ")
        k=k+1
        if k%5==0:
            print("\n")





Solve_ABCPath()









KeyError: None

  Using cached https://files.pythonhosted.org/packages/3d/a9/475622403c0a95d97c7cc84277343036940d9c061d9d02222388d9235b26/PySimpleGUI-4.5.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


  Using cached https://files.pythonhosted.org/packages/2d/33/3ae6d9d2ac8c7068937af6372fd8828ac605e62a8b17106fe57110930d38/PuLP-1.6.10.zip
  Created wheel for pulp: filename=PuLP-1.6.10-cp37-none-any.whl size=12269907 sha256=307ce390663481dcb16433e3c3df41c50d3d72f3daedd7bc31be1c2084548489
  Stored in directory: C:\Users\Krisil\AppData\Local\pip\Cache\wheels\5e\76\77\e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp
Note: you may need to restart the kernel to use updated packages.
